In [1]:
import os
import numpy as np

from silx.io.dictdump import h5todict, dicttoh5

In [2]:
cm3_to_m3 = 1e-6
ARR_PARAM = ("A", "beta", "Ta")

Inputs

In [ ]:
T = None
use_arrhenius = True
nb_lev = 9390
kin_file = "/home/zanardi/Workspace/AirDatabase/HDF5/QCT/Orig/N3_NASA_ORIG.hdf5"
arr_fit = "Arrhenius_T7500_T30000"

In [4]:
# Save path
out_path = "./../"
os.makedirs(out_path, exist_ok=True)

Kinetics database

In [5]:
kin_dtb = {}
kinetics = h5todict(kin_file)

> Molecule - Atom collisions

In [6]:
kin_dtb["m-a"] = {}

In [ ]:
if use_arrhenius:

  for (k, rates) in kinetics[arr_fit].items():
    if (k == "Diss"):
      p = "d"
      shape = (nb_lev,1,1,1,1)
    elif (k == "Inel"):
      p = "e"
      shape = (nb_lev,1,nb_lev,1)
      # Remove endothermic processes
      triu_i = np.triu_indices(nb_lev)
      for k1 in ARR_PARAM:
        rates[k1][triu_i[0],triu_i[1]] = 0.0
    else:
      continue
    # Reshape and rescale
    for k1 in ARR_PARAM:
      rates[k1] = rates[k1].reshape(shape)
    rates["A"] = rates["A"] * cm3_to_m3
    # Store
    kin_dtb["m-a"][p] = rates

else:

  for Ti in T:
    T_str = str(int(Ti))
    ma_dtb = kinetics[f"T_{T_str}_{T_str}"]["Rates"]
    # Loop over processes
    for (k, rates) in ma_dtb.items():
      if (k == "Diss"):
        p = "d"
        shape = (nb_lev,1,1,1,1)
      elif (k == "Inel"):
        p = "e"
        shape = (nb_lev,1,nb_lev,1)
        # Remove endothermic processes
        triu_i = np.triu_indices(nb_lev)
        rates[triu_i[0],triu_i[1]] = 0.0
      else:
        continue
      # Reshape and rescale
      rates = rates.reshape(shape) * cm3_to_m3
      # Store
      if (p not in kin_dtb["m-a"]):
        kin_dtb["m-a"][p] = {}
      kin_dtb["m-a"][p][T_str] = rates

> Save database

In [8]:
dicttoh5(kin_dtb, out_path + "/kinetics.hdf5")